# SVM Ejercicio - Hiperplano de Margen Máximo - Casos Kernels

Ejercicios basados en el libro Python Data Science Handbook de J.VanderPlas.

https://github.com/jakevdp/PythonDataScienceHandbook




In [1]:
import sklearn
from sklearn import datasets
#from sklearn.model_selection import train_test_split
from sklearn import svm
#from sklearn import metrics

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # biblioteca para visualización estadística.

# Caso Kernel Separable:

In [2]:
N=1000  # total de datos a generar.
#noisy_data = sklearn.datasets.make_blobs(n_samples=N, n_features=2, centers=[(-4,-4),(4,4)], cluster_std=1.0, random_state=10)  
noisy_data = sklearn.datasets.make_circles(n_samples=N, noise=0.1, factor= 0.1, random_state=7)   # factor: Scale factor between inner and outer circle in (0,1) 

In [ ]:
X, Y = noisy_data
Y = Y.reshape(Y.shape[0],1)
print(X.shape)
print(Y.shape)

In [ ]:
print(X[0:6,:])

In [ ]:
print(Y[0:6,0])

In [6]:
sns.set(rc={'figure.figsize':(9,7)})   # (width, height) - definiendo el tamaño de las gráficas.

Podríamos tratar de resolver el problema mediante un conjunto de funciones lineales... aunque no es lo más adecuado en este caso:

In [ ]:
plt.scatter(X[:,0], X[:,1], c=Y, s=20, cmap=plt.cm.Spectral);  # Puntos de ambas clases

xhs = np.linspace(-1.0, 1.0)   # Algunas rectas-clasificadores lineales: Hiperplano/margen Separador
for m, b in [(-1, 0.7),  (-0.2, -0.5), (1, -0.7), (2, 0.9)]:
    yhs = m * xhs + b
    plt.plot(xhs, yhs, '-k');

plt.plot([0], [0.6], 'x', color='green',  markersize=10);  # Un punto a clasificar

Se pueden visualizar los puntos 2D desde una persepctiva 3D al aumentar la dimensión de los datos, en este caso podemos suponer un comportamiento tipo gaussiano (simulando una montaña, por ejemplo) mediante una altura "r":

### Widget:

In [ ]:
from mpl_toolkits import mplot3d
from ipywidgets import interact, fixed

r = np.exp(-(X ** 2).sum(1))

def plot_3D(elev=30, azim=30, X=X, y=Y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap=plt.cm.Spectral)
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

interact(plot_3D, elev=[-90, 0, 20, 30, 45, 60, 70, 90], azip=(-180, 180), X=fixed(X), y=fixed(Y));

### Dibujemos algunas rectas que separen ambas clases:

In [9]:
def plot_svc_decision_function(model, ax=None, plot_support=True, alfa=0.4):    
    
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=400, facecolors='yellow', alpha=alfa);
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
modelk = svm.SVC(kernel='rbf', C=1E10)
modelk.fit(X, Y)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=Y, s=50, cmap=plt.cm.Spectral)
plot_svc_decision_function(modelk)
plt.scatter(modelk.support_vectors_[:, 0], modelk.support_vectors_[:, 1], s=300, lw=1, facecolors='none');

Veamos las coordenadas de los vectores de soporte resultantes:

In [ ]:
modelk.support_vectors_

# Caso: Kernel - No Separable

Veamos ahora el caso no-separable, donde requeriremos de funciones de kernel, en particular veamos el kernel radial gaussiano.

Recuerda que en estos casos requerimos del parámetro C asociado a las variables de holgura del método SVM, así como el parámetro gamma asociado a la desviación estándar de la gaussiana.

In [ ]:
N=1000
noisy_data2 = sklearn.datasets.make_circles(n_samples=N, noise=0.2, factor= 0.1, random_state=7)
X2, Y2 = noisy_data2
Y2 = Y2.reshape(Y2.shape[0],1)
plt.scatter(X2[:,0], X2[:,1], c=Y2, s=20, cmap=plt.cm.Spectral);

In [ ]:
#CC, gg = 10E6, 0.01
#CC, gg = 100, 0.01
CC, gg = 100, 10
#CC, gg = 20, 10
#CC, gg = 20, 80
modelnolk = svm.SVC(kernel='rbf', C=CC, gamma=gg)
modelnolk.fit(X2, Y2)

In [16]:
sns.set(rc={'figure.figsize':(12,10)}) 

In [ ]:
plt.scatter(X2[:, 0], X2[:, 1], c=Y2, s=50, cmap=plt.cm.Spectral)
plot_svc_decision_function(modelnolk, alfa=0.4)